# Агрегатные функции 
## Юнит 5. РАБОТА С БАЗАМИ ДАННЫХ. SQL
### Skillfactory: DSPR-19

### 2.1. Основные агрегатные функции
 
Как и большинство других серверов реляционных баз данных, Postgres, которую мы используем в этом тренажере, поддерживает агрегатные функции. Она позволяет вычислить единственное значение, обработав множество строк.

Например, есть агрегатные функции, вычисляющие:

- count (количество не пустых значений),
- sum (сумму),
- avg (среднее),
- max (максимум) и
- min (минимум) для набора строк.


In [ ]:
select max(book_average_rating) as max_rating 
from books;

Запрос выдал нам лишь одну строку — максимальное значение колонки book_average_rating из таблицы books. Агрегатные функции используются, когда нужно посчитать параметры, общие для всех строк таблицы. 

In [ ]:
select max(book_average_rating) as max_rating
     , min(book_average_rating) as min_rating
     , avg(book_average_rating) as average_rating
     , sum(book_ratings_count) as books_ratings
from books;

Предположим, нам нужно вычислить максимальный, минимальный и средний рейтинг книг в таблице, а также сумму количества оценок всех книг и количество самих книг, но только для книг, language_code которых равен 'eng':



In [ ]:
select max(book_average_rating) as max_rating
     , min(book_average_rating) as min_rating
     , avg(book_average_rating) as average_rating
     , sum(book_ratings_count) as books_ratings
     , count(book_id) as books_count
from books
where language_code = 'eng';

Агрегатным функциям можно также передавать не просто столбцы таблиц, но и их арифметические комбинации:

In [ ]:
select avg(book_average_rating*book_average_rating + 5) as strange_rating
from books
where language_code = 'eng';

**Важно: агрегатные функции min, max, count можно использовать и для строковых типов данных, и для даты-времени.** 

С агрегатами можно работать так же, как и с обычными столбцами. Например, их можно перемножать или делить. Предположим, мы хотим получить среднее арифметическое по столбцу book_average_rating. Для этого мы можем просто использовать агрегатную функцию avg, а можем поделить сумму рейтингов на их количество:

Для подсчёта количества непустых строк можно использовать count(*):

In [ ]:
select count(*) not_null_strings_count
from books

### Задание 2.1.1
Напишите запрос, который находит максимальный book_id и размещает его в столбце с именем max_id (... as max_id) среди книг, у которых рейтинг меньше 4.

In [ ]:
select max(book_id) as max_id 
from books
where book_average_rating < 4

### Задание 2.1.2
Определите средний рейтинг (as average_rating) и минимальное количество оценок книги (min_ratings) для книг Стивена Кинга (Stephen King), изданных позднее 1990 года. Укажите запрос ниже:

In [ ]:
select avg(book_average_rating) as average_rating, min(book_ratings_count) as min_rating
from books
where author = 'Stephen King' and publishing_year > 1990

### Задание 2.1.3
Подсчитайте количество непустых строк в столбце book_name (не изменяйте название столбца) среди всех книг с рейтингом < 4 и количеством оценок > 100000.

In [ ]:
select count(book_name)
from books
where book_average_rating <4 and book_ratings_count > 100000

### Задание 2.1.4
Выведите минимальное значение столбца book_name для книг, у которых (language_code = 'eng' или language_code = 'en-US' и больше 10000 оценок), либо у которых (жанр 'fiction' или год публикации 1957). Скобки в условии указывают порядок действий.

In [ ]:
select min(book_name)
from books
where ((language_code = 'eng' or language_code = 'en-US') and book_ratings_count > 10000) or (genre = 'fiction' or publishing_year = 1957 )

### 2.2. DISTINCT и GROUP BY

Иногда нужны не все данные из таблицы, а только уникальные значения в столбце. Например, мы хотим получить всех авторов книг в таблице books. Для этого может использоваться команда DISTINCT:

In [ ]:
select distinct author
from books
order by author;

DISTINCT можно применять не только к одному столбцу, но и к нескольким сразу, тогда будут получены все уникальные комбинации столбцов в этих строках:

In [ ]:
select distinct author, publishing_year
from books
order by author, publishing_year;

DISTINCT можно применять не только для вывода уникальных значений, но и внутри агрегатных функций для того, чтобы они применялись только к уникальным элементам определённых столбцов. Например, если мы хотим посчитать количество авторов в нашей базе, то можно написать: 

In [ ]:
select count(distinct author) authors_count
from books;

### Задание 2.2.1
Сколько уникальных наименований книг содержится в базе?

Ответ: 1045

In [ ]:
select count(distinct book_name) book_name_count
from books;